# 모델링

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl
from datetime import datetime
from datetime import timedelta

# Windows
mpl.rc("font", family='Malgun Gothic')

warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
csv_EDA = 'C:/Users/JEON_SANGEON/codestates/data/cp2_EDA.csv'
rating = 'C:/Users/JEON_SANGEON/codestates/data/view_rating.csv'

df_EDA= pd.read_csv(csv_EDA)


In [3]:
df_rating = pd.read_csv(rating)

In [4]:
# csv 불러와서 날짜컬럼 재설정
df_EDA['event_time'] = pd.to_datetime(df_EDA['event_time'], utc=True)
df_EDA['event_time'] = df_EDA['event_time'].dt.tz_convert('Asia/Dubai')

In [5]:
# 불필요 컬럼 제거
del df_EDA['Unnamed: 0']

In [6]:
df_EDA = df_EDA.loc[(df_EDA['price']!=0)]

In [7]:
df = df_EDA.copy()

In [8]:
del df_rating['Unnamed: 0']

df_rating

,user_id,category_code,value
0,265601964,accessories.bag,0.142857
1,301823874,accessories.bag,0.083333
2,338504180,accessories.bag,1.000000
3,388529847,accessories.bag,0.250000
4,414404621,accessories.bag,0.666667
...,...,...,...
6989139,566109860,stationery.cartrige,1.000000
6989140,566122879,stationery.cartrige,1.000000
6989141,566127538,stationery.cartrige,1.000000
6989142,566132392,stationery.cartrige,0.333333


In [9]:
df_view = df.loc[df['event_type']=='view']

In [10]:
# merge

df = pd.merge(left=df_view, right=df_rating,how='left',\
                    on=['user_id','category_code'],sort=False)

In [11]:
df


,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,value
0,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,44600062,2103807459595387724,no_cat,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,0.672000
1,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,1.000000
2,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,17200506,2053013559792632471,furniture.living_room.sofa,no_brand,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,0.047619
3,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,0.318182
4,2019-10-01 04:00:04+04:00,10,1,Tuesday,4,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,0.306818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40710871,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193,0.147059
40710872,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,10800172,2053013554994348409,appliances.kitchen.multibaker,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8,0.081967
40710873,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd,1.000000
40710874,2019-11-01 03:59:59+04:00,11,1,Friday,3,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204,0.565217


참조 : 
https://github.com/journey101/Recommendation_CF_Hybrid_Modeling_with_Python_Surprise/blob/main/Recommendation_CF_Hybrid_modeling_for_Fashion_update(210621).ipynb

In [10]:
df

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,44600062,2103807459595387724,no_cat,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,17200506,2053013559792632471,furniture.living_room.sofa,no_brand,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 04:00:04+04:00,10,1,Tuesday,4,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42448759,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448760,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,10800172,2053013554994348409,appliances.kitchen.multibaker,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
42448761,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd
42448762,2019-11-01 03:59:59+04:00,11,1,Friday,3,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204


In [13]:
item_ = df.groupby(by='product_id').count()
item_

,event_time,month,day,day_name,hour,event_type,category_id,category_code,brand,price,user_id,user_session
product_id,,,,,,,,,,,,
1000978,3094,3094,3094,3094,3094,3094,3094,3094,3094,3094,3094,3094
1001588,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276,2276
1001606,16,16,16,16,16,16,16,16,16,16,16,16
1002042,328,328,328,328,328,328,328,328,328,328,328,328
1002062,1574,1574,1574,1574,1574,1574,1574,1574,1574,1574,1574,1574
...,...,...,...,...,...,...,...,...,...,...,...,...
60500005,14,14,14,14,14,14,14,14,14,14,14,14
60500006,14,14,14,14,14,14,14,14,14,14,14,14
60500007,11,11,11,11,11,11,11,11,11,11,11,11


In [14]:
user_ = df.groupby(by='user_id').count()
user_

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_session
user_id,,,,,,,,,,,,
33869381,1,1,1,1,1,1,1,1,1,1,1,1
183503497,1,1,1,1,1,1,1,1,1,1,1,1
184265397,6,6,6,6,6,6,6,6,6,6,6,6
195082191,1,1,1,1,1,1,1,1,1,1,1,1
200673532,4,4,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
566280663,2,2,2,2,2,2,2,2,2,2,2,2
566280676,1,1,1,1,1,1,1,1,1,1,1,1
566280697,1,1,1,1,1,1,1,1,1,1,1,1


In [15]:
user_.sort_values(by="product_id", ascending=False)

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_session
user_id,,,,,,,,,,,,
512475445,7429,7429,7429,7429,7429,7429,7429,7429,7429,7429,7429,7429
512365995,4012,4012,4012,4012,4012,4012,4012,4012,4012,4012,4012,4012
526731152,2912,2912,2912,2912,2912,2912,2912,2912,2912,2912,2912,2912
512505687,2894,2894,2894,2894,2894,2894,2894,2894,2894,2894,2894,2894
513021392,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862
...,...,...,...,...,...,...,...,...,...,...,...,...
554255592,1,1,1,1,1,1,1,1,1,1,1,1
554255594,1,1,1,1,1,1,1,1,1,1,1,1
554255636,1,1,1,1,1,1,1,1,1,1,1,1


In [16]:
df.groupby(by='category_code').count()

,event_time,month,day,day_name,hour,event_type,product_id,category_id,brand,price,user_id,user_session
category_code,,,,,,,,,,,,
accessories.bag,190017,190017,190017,190017,190017,190017,190017,190017,190017,190017,190017,190017
accessories.jewelry,1672378,1672378,1672378,1672378,1672378,1672378,1672378,1672378,1672378,1672378,1672378,1672378
accessories.umbrella,3923,3923,3923,3923,3923,3923,3923,3923,3923,3923,3923,3923
accessories.wallet,43476,43476,43476,43476,43476,43476,43476,43476,43476,43476,43476,43476
apparel.belt,2429,2429,2429,2429,2429,2429,2429,2429,2429,2429,2429,2429
...,...,...,...,...,...,...,...,...,...,...,...,...
sport.ski,3005,3005,3005,3005,3005,3005,3005,3005,3005,3005,3005,3005
sport.snowboard,3727,3727,3727,3727,3727,3727,3727,3727,3727,3727,3727,3727
sport.tennis,3540,3540,3540,3540,3540,3540,3540,3540,3540,3540,3540,3540


product_id별 user_id : 1:n관계 \
user_id별 product_id : 1:n관계 \
category_code 별 product_id : 1:n관계 

추천 시스템에 적용하기에 적합한 데이터로 판단됨. (유사 유저그룹 판단 후 아직 경험하지 않은 아이템을 추천해야 하기 때문)

In [12]:
df = df.head(100000)

# 아래는 원본 데이터로 진행된 내용

In [13]:
# rating을 value로 놓고 진행함.


mean_df = df[['product_id','value']].groupby(by='product_id').mean()
mean_df = mean_df.reset_index()
mean_df = mean_df.rename(columns={'value': 'rating_average'})
mean_df

,product_id,rating_average
0,1001588,0.815028
1,1002062,1.000000
2,1002098,0.719937
3,1002099,0.768884
4,1002100,0.659242
...,...,...
20910,52900101,0.141447
20911,53100005,0.090909
20912,53100006,0.176471
20913,53100013,0.155738


In [14]:
df_r_m = pd.merge(df, mean_df, how='left', on='product_id')
df_r_m['user_id'] = df_r_m['user_id'].astype(int)
df_r_m['product_id'] = df_r_m['product_id'].astype(int)
# df_r_merge.dtypes

df_r_m.head()

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,value,rating_average
0,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,44600062,2103807459595387724,no_cat,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,0.672000,0.672000
1,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,1.000000,0.265464
2,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,17200506,2053013559792632471,furniture.living_room.sofa,no_brand,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,0.047619,0.463999
3,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,0.318182,0.692627
4,2019-10-01 04:00:04+04:00,10,1,Tuesday,4,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,0.306818,0.753152


In [23]:
df_r_m.dtypes

event_time        datetime64[ns, Asia/Dubai]
month                                  int64
day                                    int64
day_name                              object
hour                                   int64
event_type                            object
product_id                             int32
category_id                            int64
category_code                         object
brand                                 object
price                                float64
user_id                                int32
user_session                          object
value                                float64
rating_average                       float64
dtype: object

In [25]:
df_r_m['value'].value_counts()

1.000000    7353892
0.500000     737370
0.333333     480161
0.666667     468136
0.250000     358332
             ...   
0.001715          1
0.000455          1
0.001131          1
0.000904          1
0.001812          1
Name: value, Length: 33765, dtype: int64

# 모델 분석방법

surprise 모듈 임포트

In [15]:
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise import accuracy
from surprise import dump
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [16]:
# train, test set split 

## data randomly shuffling.  
df_r_m = df_r_m.sample(frac=1)
df_r_m

len(df_r_m)
cutoff = int(.5 * len(df_r_m))
train_r = df_r_m[:cutoff]
test_r = df_r_m[cutoff:]

train_r = train_r[['user_id', 'product_id', 'value', 'category_code', 'brand']]
test_r = test_r[['user_id', 'product_id', 'value', 'category_code', 'brand']]
print("train_size:", len(train_r), "test_size:", len(test_r))
train_r.head()

train_size: 50000 test_size: 50000


,user_id,product_id,value,category_code,brand
97490,548707010,4100214,0.062500,electronics.video.tv,nintendo
50677,513414569,3601437,0.128492,appliances.kitchen.dishwasher,lg
88266,514534397,6200978,0.099391,appliances.environment.air_heater,no_brand
63027,513033464,26000565,0.833333,missing,missing
62696,549395480,17200570,0.875000,furniture.living_room.sofa,no_brand


In [17]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(0, 5))
    trainset = Dataset.load_from_df(training_dataframe[['user_id', 'product_id', 'value']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['user_id', 'product_id', 'value']], reader)
    
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)
    
    return trainset, testset

In [18]:

trainset, testset = convert_traintest_dataframe_forsurprise(train_r, test_r)

## (1) Knn-baseline 모델

In [19]:
# basic collaborative filtering algorithm taking into account a baseline rating.
# content-based filtering (CBF)
# 참조: https://surprise.readthedocs.io/en/stable/knn_inspired.html

sim_options = {'name': 'cosine',
               'user_based': False  # content(item)-based similarity
               }
knnbaseline_algo_i = KNNBaseline(sim_options=sim_options)

knnbaseline_algo_i.fit(trainset)
knnbaseline_predictions_i = knnbaseline_algo_i.test(testset)

file_name = 'KnnBaseline_model_i'
dump.dump(file_name, algo=knnbaseline_predictions_i)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(knnbaseline_predictions_i)
accuracy.mae(knnbaseline_predictions_i)
print("Done!")

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.2211
MAE:  0.1529
Done!


In [20]:
# basic collaborative filtering algorithm taking into account a baseline rating.
# user-based filtering (CBF)
# 참조: https://surprise.readthedocs.io/en/stable/knn_inspired.html

sim_options = {'name': 'cosine',
               'user_based': True  # user-based similarity
               }
knnbaseline_algo_u = KNNBaseline(sim_options=sim_options)

knnbaseline_algo_u.fit(trainset)
knnbaseline_predictions_u = knnbaseline_algo_u.test(testset)

file_name = 'KnnBaseline_model_u'
dump.dump(file_name, algo=knnbaseline_predictions_u)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(knnbaseline_predictions_u)
accuracy.mae(knnbaseline_predictions_u)
print("Done!")

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.2673
MAE:  0.2143
Done!


In [21]:
knnbaseline_predictions_i = knnbaseline_algo_i.test(testset)
knnbaseline_predictions_i[:3]

[Prediction(uid=544371892, iid=22700067, r_ui=0.2333333333333333, est=0.2059819315766458, details={'actual_k': 3, 'was_impossible': False}),
 Prediction(uid=515122879, iid=5100567, r_ui=0.1304347826086956, est=0.5553483785315021, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=514680524, iid=4200882, r_ui=1.0, est=0.5594184235648336, details={'was_impossible': False})]

In [22]:
# uid, iid, r_ui(실제 평점) ,est(예측평점) 추출

[(pred.uid, pred.iid, pred.est) for pred in knnbaseline_predictions_i[:3]]

[(544371892, 22700067, 0.2059819315766458),
 (515122879, 5100567, 0.5553483785315021),
 (514680524, 4200882, 0.5594184235648336)]

In [24]:
# uid = 998923 에 대한 유사도 높은 예측 iid, est 조회. 
# 참고: https://hmkim312.github.io/posts/%EC%84%9C%ED%94%84%EB%9D%BC%EC%9D%B4%EC%A6%88(Surprise)%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%9C_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C(Recommendations)/

pred_df = pd.DataFrame(knnbaseline_predictions_i)
pred_df[(pred_df['uid']==544371892)]

,uid,iid,r_ui,est,details
0,544371892,22700067,0.233333,0.205982,"{'actual_k': 3, 'was_impossible': False}"
32163,544371892,22700069,0.233333,0.218720,"{'actual_k': 2, 'was_impossible': False}"
40299,544371892,22700068,0.233333,0.220167,"{'actual_k': 4, 'was_impossible': False}"


## (2) svd, svdpp (CF) 모델

In [25]:
 ## svd 
 ## 참고: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#matrix-factorization-based-algorithms

svd_algo = SVD()

svd_algo.fit(trainset)
svd_predictions = svd_algo.test(testset)

file_name = 'svd_model'
dump.dump(file_name, algo=svd_algo)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(svd_predictions)
accuracy.mae(svd_predictions)
print("Done!")

RMSE: 0.2516
MAE:  0.2021
Done!


In [26]:
# matrix factorization/ collaborative filtering(CF)
 ## svd++ model : taking into account implicit ratings 
 ## 참고: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp

svdpp_algo = SVDpp()

svdpp_algo.fit(trainset)
svdpp_predictions = svdpp_algo.test(testset)

file_name = 'svdpp_model'
dump.dump(file_name, algo=svdpp_algo)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(svdpp_predictions)
accuracy.mae(svdpp_predictions)
print("Done!")

RMSE: 0.2365
MAE:  0.1875
Done!


--여기까지 진행됨.

원본 데이터로 진행시 메모리 부족으로 진행이 안되었음.

10만개로 축소하고 진행됨.

## (3) gridsearchCV 모델

In [28]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [5, 20], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
              # lr_all – The learning rate for all parameters. Default is 0.005.
              # reg_all – The regularization term for all parameters. Default is 0.02.
              
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5) 

reader = Reader()

# The columns must correspond to user id, item id and ratings (in that order).
train_data = Dataset.load_from_df(train_r[['user_id', 'product_id', 'value']], reader)

gs.fit(train_data)

# best RMSE score
print("gs.best_score(rmse): ", gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print("gs.best_params: ", gs.best_params['rmse'])

gs.best_score(rmse):  0.5636409335453909
gs.best_params:  {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}


In [29]:
from surprise import accuracy

# best model fit 
# We can now use the algorithm that yields the best rmse:
GS_SVDpp_best = gs.best_estimator['rmse']

# retrain on the whole set train
trainset = train_data.build_full_trainset()
GS_SVDpp_best.fit(trainset)

# Compute biased accuracy on train (validation set)
predictions = GS_SVDpp_best.test(trainset.build_testset())
print('validation test,', end='\n')
accuracy.rmse(predictions)
accuracy.mae(predictions)

# 참고:https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-save-some-data-for-unbiased-accuracy-estimation
## 참고: https://surprise.readthedocs.io/en/stable/getting_started.html#train-on-whole-trainset

validation test,
RMSE: 0.5637
MAE:  0.4406


0.44058157643516643

In [30]:

# Compute biased accuracy on train (validation set)
predictions = GS_SVDpp_best.test(testset)
print('testset accuracy,', end='\n')
accuracy.rmse(predictions)
accuracy.mae(predictions)

testset accuracy,
RMSE: 0.5651
MAE:  0.4429


0.4428538942453815

In [35]:
# k-fold trainset, testset 만드는 함수 
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import KFold

reader = Reader()
data = Dataset.load_from_df(df[['user_id', 'product_id', 'value']], reader)

# define a cross-validation iterator
kf = KFold(n_splits=5)

def train_test_get(data, gettype):
  for trainset, testset in kf.split(data):
    if gettype == "train":
      return trainset
    elif gettype == "test":
      return testset 

train_test_get(data, "test")

[(517676026, 26200395, 0.7931034482758621),
 (539464221, 28714424, 0.303030303030303),
 (513447975, 6301259, 0.7),
 (513632293, 1004958, 1.0),
 (512392338, 1002524, 0.7674418604651163),
 (537424879, 19200049, 0.9705882352941176),
 (513005003, 8500122, 0.925925925925926),
 (525457519, 1004777, 0.7777777777777778),
 (512597715, 27700024, 0.0217391304347826),
 (548014274, 37900169, 0.0714285714285714),
 (516379318, 6000248, 0.8205128205128205),
 (534816139, 1004741, 0.75),
 (550284046, 28711793, 0.3734177215189873),
 (512756464, 1004144, 0.9682539682539684),
 (521805127, 1004375, 1.0),
 (543103721, 34200027, 0.369047619047619),
 (513859714, 5301408, 0.8714285714285714),
 (552982808, 5100337, 0.0227272727272727),
 (546241966, 1005033, 0.2),
 (512512173, 26400182, 0.5),
 (513674690, 2702023, 0.1092436974789916),
 (539432849, 2601860, 1.0),
 (513840435, 1004288, 1.0),
 (545070506, 1004659, 1.0),
 (514074741, 8800599, 0.0434782608695652),
 (529408483, 7004322, 0.2),
 (516406369, 47600000, 0.2

In [36]:

# testset 
pred_testset = algo_best.test(train_test_get(data, "test"))

NameError: name 'algo_best' is not defined

In [34]:
# cv_results 
results = pd.DataFrame.from_dict(gs.cv_results)
results.T

,0,1,2,3,4,5,6,7
split0_test_rmse,0.564344,0.564344,0.564344,0.564344,0.564344,0.564344,0.564344,0.564344
split1_test_rmse,0.559663,0.559663,0.559663,0.559663,0.559663,0.559663,0.559663,0.559663
split2_test_rmse,0.561857,0.561857,0.561857,0.561857,0.561857,0.561857,0.561857,0.561857
split3_test_rmse,0.562862,0.562862,0.562862,0.562862,0.562862,0.562862,0.562862,0.562862
split4_test_rmse,0.569479,0.569479,0.569479,0.569479,0.569479,0.569479,0.569479,0.569479
mean_test_rmse,0.563641,0.563641,0.563641,0.563641,0.563641,0.563641,0.563641,0.563641
std_test_rmse,0.003292,0.003292,0.003292,0.003292,0.003292,0.003292,0.003292,0.003292
rank_test_rmse,1,2,3,4,5,6,7,8
split0_test_mae,0.440564,0.440564,0.440564,0.440564,0.440564,0.440564,0.440564,0.440564
split1_test_mae,0.436173,0.436173,0.436173,0.436173,0.436173,0.436173,0.436173,0.436173


## 1-2) evaluation (knn/svd/svdpp/gridsearchCV 비교)

- knnbaseline (item_based)
- knnbaseline (user_based)
- SVD (CF)
- SVDpp (CF)
- Gridsearch (CF best)
- recall 비교: evaluating_recs.py
- nDCG 개념

### prediction, rmse, mae, precision, recall, f-measure, ndcg_score 계산값

In [ ]:
# predictions 
knn_i_pred = knnbaseline_algo_i.test(testset)
knn_u_pred = knnbaseline_algo_u.test(testset)
svd_pred = svd_algo.test(testset)
svdpp_pred = svdpp_algo.test(testset)
gridCV_svdpp_pred = algo_best.test(testset)

In [ ]:
# test_rmse, test_mae
knn_i_rmse = accuracy.rmse(knn_i_pred)
knn_i_mae = accuracy.mae(knn_i_pred)
knn_u_rmse = accuracy.rmse(knn_u_pred)
knn_u_mae = accuracy.mae(knn_u_pred)
svd_rmse = accuracy.rmse(svd_pred)
svd_mae = accuracy.mae(svd_pred)
svdpp_rmse = accuracy.rmse(svdpp_pred)
svdpp_mae = accuracy.mae(svdpp_pred)
gridCV_svdpp_rmse = accuracy.rmse(gridCV_svdpp_pred)
gridCV_svdpp_mae = accuracy.mae(gridCV_svdpp_pred)

In [ ]:
# recall & precision @ k function
# 참고: https://surprise.readthedocs.io/en/stable/FAQ.html?highlight=recall#how-to-compute-precision-k-and-recall-k

from collections import defaultdict
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [ ]:
# knn_i 모델 precision&recall
knn_i_prec, knn_i_re = precision_recall_at_k(knn_i_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
knn_i_prec_avg = sum(prec for prec in knn_i_prec.values()) / len(knn_i_prec)
print(knn_i_prec_avg)
knn_i_re_avg = sum(rec for rec in knn_i_re.values()) / len(knn_i_re)
print(knn_i_re_avg)

In [ ]:
# knn_u 모델 precision&recall
knn_u_prec, knn_u_re = precision_recall_at_k(knn_u_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
knn_u_prec_avg = sum(prec for prec in knn_u_prec.values()) / len(knn_u_prec)
print(knn_u_prec_avg)
knn_u_re_avg = sum(rec for rec in knn_u_re.values()) / len(knn_u_re)
print(knn_u_re_avg)

In [ ]:
# svd precision&recall
svd_prec, svd_re = precision_recall_at_k(svd_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
svd_prec_avg = sum(prec for prec in svd_prec.values()) / len(svd_prec)
print(svd_prec_avg)
svd_re_avg = sum(rec for rec in svd_re.values()) / len(svd_re)
print(svd_re_avg)

In [ ]:
# svdpp precision&recall
svdpp_prec, svdpp_re = precision_recall_at_k(svdpp_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
svdpp_prec_avg = sum(prec for prec in svdpp_prec.values()) / len(svdpp_prec)
print(svdpp_prec_avg)
svdpp_re_avg = sum(rec for rec in svdpp_re.values()) / len(svdpp_re)
print(svdpp_re_avg)

In [ ]:
# gridCV_svdpp precision&recall

grid_prec, grid_re = precision_recall_at_k(gridCV_svdpp_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
grid_prec_avg = sum(prec for prec in grid_prec.values()) / len(grid_prec)
print(grid_prec_avg)
grid_re_avg = sum(rec for rec in grid_re.values()) / len(grid_re)
print(grid_re_avg)

### model별 test_rmse/mae/precision/recall 테이블

In [ ]:
# model별 test prediction  데이터 
# row: Knnbaseline(item based), Knnbaseline(user based), SVD(CF), SVDpp(CF), gridsearch(SVDpp)
# columns: test RMSE, test MAE, test_time, Precision, recall
# hybrid 모델 수치는 SVDpp(CF)모델과 Knnbaseline(item-based)의 평균값. 
test_prediction = pd.DataFrame({"Algorithm": ["KnnBaseline(item-based)" ,  
                                              "KnnBaseline(user-based)", 
                                              "SVD(CF)",
                                              "SVDpp(CF)",
                                              "gridsearch(SVDpp)",
                                              "Hybrid(CF+CBF)"

                                              ]
                                ,"test_rmse": [knn_i_rmse , knn_u_rmse, svd_rmse , svdpp_rmse , gridCV_svdpp_rmse, svdpp_rmse*0.4+knn_i_rmse*0.6]
                                ,"test_mae":[knn_i_mae , knn_u_mae, svd_mae, svdpp_mae , gridCV_svdpp_mae, svdpp_mae*0.4+knn_i_mae*0.6]
                                ,"Precision":[knn_i_prec_avg ,knn_u_prec_avg ,svd_prec_avg ,svdpp_prec_avg , grid_prec_avg, svdpp_prec_avg*0.4+knn_i_prec_avg*0.6]
                                ,"recall":[knn_i_re_avg ,knn_u_re_avg ,svd_re_avg ,svdpp_re_avg ,grid_re_avg, svdpp_re_avg*0.4+knn_i_re_avg*0.6 ]
                                })

test_prediction

## 1-3) 추천목록 비교

- (CF vs Hybrid_latent feature added_model 목록)

(1) 콘텐츠필터링/협업필터링 모델의 top N 추천목록 (※주의: top-n 구하는 코드 시간이 오래걸림)

In [ ]:
# 참고: https://www.fun-coding.org/recommend_basic7.html

# 1안) n=5로 줄여서 진행. 
# 2안) 원데이터 자체를 1만까지 줄여서 진행. 

from collections import defaultdict
from surprise import Dataset
from surprise import accuracy

# top k item list 구하는 함수  
def get_top_n(predictions, n):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
# svdpp_pred
testset = trainset.build_anti_testset()
# knn_i_pred = knnbaseline_algo_i.test(testset)
svdpp_pred = svdpp_algo.test(testset)

  # 참고: https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-save-some-data-for-unbiased-accuracy-estimation
  # 참고: https://surprise.readthedocs.io/en/stable/algobase.html#surprise.prediction_algorithms.algo_base.AlgoBase.predict
  # 참고: https://surprise.readthedocs.io/en/stable/FAQ.html#what-are-raw-and-inner-ids

svdpp_top_n = get_top_n(svdpp_pred, n=5)

In [ ]:
svdpp_top_n_df = pd.DataFrame(svdpp_top_n)
# knn_i_top_n_df.groupby(by=222811).max()
result = svdpp_top_n_df[222811].values

In [ ]:
svdpp_test = test_r[['user_id', 'item_id', 'rating']][ ( test_r['item_id']==123793 ) | ( test_r['item_id']==138431 ) | ( test_r['item_id']==730008 ) | 
                             ( test_r['item_id']==2918587 ) | ( test_r['item_id']==612756 )]
# knn_test = knn_test[ ( knn_test['user_id']== 222811)]
# knn_test
svdpp_test

In [ ]:
# category, brand를 보고자 한다면 이렇게 따로 볼 수는 있음. 
svdpp_top_n = df_r_m[['product_id','category_code', 'brand']][(df_r_m['product_id'] == 5100816) 
| (df_r_m['product_id'] == 12703498) |(df_r_m['product_id'] == 12703015) |(df_r_m['product_id'] == 22700084)
].drop_duplicates()
# knn_top_n['est'] = top_n
# knn_top_n.groupby(by='item_id').count()
svdpp_top_n.head()

## (2) 📍hybrid(CF(SVDpp)+CBF(Content-Based or Item-Based Filtering)+Item-Similiarity+Popularity)

- (CF+CBF 4:6가중평균합 모델) + (Item-Similarity모델) + (Popularity 모델)을 혼합한 모델.

### (2-1) item similarity 모델 (category기준)
- 하이브리드 모델에 들어갈 Item-Similarity 모델

In [ ]:

category = df_r_m['category_code'].unique().tolist()
category

In [ ]:
category.index('computers.notebook')

In [ ]:
category_r_to_idx = dict()
def addcategory(category):
  for i in category: # category리스트를 돈다
    category_r_to_idx[i] = category.index(i)
  return category_r_to_idx

addcategory(category)

In [ ]:
# idx_to_category_r
  
idx_to_category_r = dict([(value, key) for key, value in category_r_to_idx.items()]) 
idx_to_category_r

In [ ]:
# category vectorizing 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf_new = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix_new_r = tf_new.fit_transform(df_r_m['category'])

In [ ]:

# category 가 추가된 matrix 의 cosine similarity 계산
cosine_sim_new = linear_kernel(tfidf_matrix_new_r, tfidf_matrix_new_r)

In [ ]:
category_name = df_r_m['category_code']
indices = pd.Series(df_r_m.index, index=df_r_m['category_code'])
indices.head(10)

In [ ]:
indices["top"]

In [ ]:

def get_recommendations_new(category_name):
    
    idx = indices[category_name]
    if type(idx) != np.int64:
        if len(idx)>1:
            print("ALERT: Multiple values")
            idx = idx[0]
    sim_scores = list(enumerate(cosine_sim_new[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    item_indices = [i[0] for i in sim_scores]
    return df_r_m['item_id'].iloc[item_indices]

# get_recommendations_new("top") # 특정 category에 대한 아이템 추천. 

### (2-2) popularity 모델 (category기준)
- 하이브리드 모델에 들어갈 popularity 모델

In [ ]:
df_r_m['rating_average'] = df_r_m['rating_average'].astype(int)
df_r_m.dtypes

In [ ]:
# popularity model (popularity = rating_average)
def category_based_popularity(category):
 
    mask = df_r_m.category.apply(lambda x: category in x)
    filtered_item = df_r_m[mask]
    filtered_item = filtered_item.sort_values(by='rating_average', ascending=False)

    return filtered_item['item_id'].head(10).values.tolist() 

category_based_popularity('top')

In [ ]:
def user_top_category(user_id):
    user_id = df_r_m['user_id'][(df_r_m['user_id'] == user_id)].values[0].copy()
    print("User vec: ", user_id)
    top_category_indices = np.flip(np.argsort(user_id))
    category_list = []
    for i in top_category_indices[:3]:
        category_list.append(idx_to_category_r[i])
    return category_list

In [ ]:
user_id = 535871217
# df_r_m['user_id'][(df_r_m['user_id'] == user_id)].values.copy()
df_r_m[(df_r_m['user_id'] == user_id)]

In [ ]:
user_top_category(535871217)